pip install nest_asyncio

git clone --depth=1 https://github.com/twintproject/twint.git<br>
cd twint<br>
pip3 install . -r requirements.txt<br>

+ aiohttp==3.7.0 setzen in requirements.txt

python 3.7

In [1]:
import twint
import pandas as pd
import datetime
import schedule
import time
import os
import nest_asyncio
nest_asyncio.apply()
pd.set_option('display.max_columns', None)

c = twint.Config()

In [2]:
# Erstellt aus String eine Liste
from ast import literal_eval
import re
def extract_reply_to(stringliste):
    if len(stringliste) > 10:
        mentionsliste = []
        liste = literal_eval(stringliste)
        for mention in liste:
            mentionsliste.append(mention["screen_name"])
        return mentionsliste
    else:
        return stringliste

In [3]:
def searchTwint(keyword, since=None, until=None, language=None, limit=None, createDataFrame=True, hide_out=True):
    c = twint.Config()
    c.Search = "{} lang:{}".format(keyword, language)
    c.Filter_retweets = False
    if language is not None:
        c.Lang = language
    if since is not None:
        c.Since = since
    if until is not None:
        c.Until = until
    if limit is not None:
        c.Limit = limit
    c.Hide_output = hide_out
    if createDataFrame:
        c.Pandas = True
    twint.run.Search(c)
    if createDataFrame:
        Tweets_df = twint.storage.panda.Tweets_df
        return Tweets_df

In [4]:
df = searchTwint("corona", since="2022-05-27", until="2022-05-29", language="de", limit=1000, createDataFrame=True)

In [5]:
# Duplikate werden entfernt
df = df.drop_duplicates(subset=["id"])
df.drop(columns=["created_at", "place", "cashtags", "user_id_str", "day", "hour", "urls", "thumbnail", "quote_url", "near", "geo", "source", "user_rt_id", "user_rt", "retweet_id", "retweet", "retweet_date", "translate", "trans_src", "trans_dest"], inplace=True)

In [6]:
df = df[df["language"] == "de"]
df["date"] = df["date"].astype("datetime64")
#df["created_at"] = df["created_at"].astype("datetime64")
df["Wochentag"] = df["date"].apply(lambda x: x.weekday())
df["Stunde"] = df["date"].apply(lambda x: x.hour)
df["Originale_Tweetlaenge"] = df["tweet"].apply(lambda x: len(x))
#df["reply_to"] = df["reply_to"].apply(lambda x: str(extract_reply_to(x)))

In [7]:
df

,id,conversation_id,date,timezone,tweet,language,hashtags,user_id,username,name,link,photos,video,nlikes,nreplies,nretweets,search,reply_to,Wochentag,Stunde,Originale_Tweetlaenge
0,1530699449323995136,1530699449323995136,2022-05-29 01:56:02,+0200,Der wahre Feind für den Bürger ist die Abwesen...,de,"[corona, demokratie, ifsg]",1530150359691411456,FionaIamBack,Fiona,https://twitter.com/FionaIamBack/status/153069...,[],0,4,0,1,corona lang:de,[],6,1,95
1,1530699272844460032,1530699272844460032,2022-05-29 01:55:20,+0200,Maskenpflicht und weitere Maßnahmen: RKI-Chef ...,de,[],326899497,Ostseeolaf,Ostseeolaf,https://twitter.com/Ostseeolaf/status/15306992...,[],0,0,2,4,corona lang:de,[],6,1,158
2,1530698754969550849,1530698754969550849,2022-05-29 01:53:16,+0200,Tanzen als ob Corona nur Fake News ist. Nobody...,de,[],1325940815592648704,GraffityJimmy,Jimmy Graffity,https://twitter.com/GraffityJimmy/status/15306...,[https://pbs.twimg.com/media/FT4iZb9XEAQuy9a.jpg],1,1,0,0,corona lang:de,[],6,1,174
3,1530697203643297792,1530596403453513728,2022-05-29 01:47:06,+0200,@_MartinHagen @c_lindner @focusonline Die @FDP...,de,"[fdprausausderregierung, fdprausausdenparlamen...",3378490043,dannyalgaaf,Danny Al-Gaaf,https://twitter.com/dannyalgaaf/status/1530697...,[],0,18,0,2,corona lang:de,"[{'screen_name': '_MartinHagen', 'name': 'Mart...",6,1,279
4,1530696675605504002,1530653076239138817,2022-05-29 01:45:01,+0200,@chguettler Wegen Corona verlieren die Wenigst...,de,[],1488993896,gunder42,aRGy,https://twitter.com/gunder42/status/1530696675...,[],0,2,0,0,corona lang:de,"[{'screen_name': 'chguettler', 'name': 'Chris'...",6,1,163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,1530597854313689089,1530526561715224578,2022-05-28 19:12:20,+0200,@karla__kraus @MeyerMeike Es ist Dummheit. Tro...,de,[],1495686600558317571,TheThickPen,The Thick Pen 💉💉😷💉💉,https://twitter.com/TheThickPen/status/1530597...,[],0,1,0,0,corona lang:de,"[{'screen_name': 'karla__kraus', 'name': '🇺🇦 K...",5,19,305
1013,1530597818393677824,1530555290600493056,2022-05-28 19:12:11,+0200,@nenacasc @annewill Anne Will hat bisher mehr ...,de,[],1355233260163489792,KirchJurgen,Jürgen Kirch,https://twitter.com/KirchJurgen/status/1530597...,[],0,2,0,0,corona lang:de,"[{'screen_name': 'nenacasc', 'name': 'NENA SCH...",5,19,175
1014,1530597736881471491,1529002686997315585,2022-05-28 19:11:52,+0200,@Karl_Lauterbach ich hab mich früher nie für P...,de,[],15944982,valshare,valshare,https://twitter.com/valshare/status/1530597736...,[],0,0,0,0,corona lang:de,"[{'screen_name': 'Karl_Lauterbach', 'name': 'P...",5,19,282
1015,1530597629201108999,1530597629201108999,2022-05-28 19:11:26,+0200,Bin immer noch nicht von #Affenpocken befallen...,de,"[affenpocken, corona, grippe, corona, geimpft,...",1433048619452612608,9Incide,Inside ♒,https://twitter.com/9Incide/status/15305976292...,[https://pbs.twimg.com/media/FT3Gbr2WIAA1JN6.jpg],1,2,1,1,corona lang:de,[],5,19,304


In [8]:
df.iloc[10]["tweet"]

'@welt Ich fordere dass mich die Corona-Freaks bitte dauerhaft in Ruhe lassen sollen, mit ihrer Paranoia. Ich glaube nicht dass ich abkratzen werde, wenn ich im Winter mal ein paar Tage einen Omikron Schnupfen habe.'

In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from typing import List
import torch
import re

class SentimentModel():
    def __init__(self, model_name):
        self.device = "cpu"
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Cleaning with Regex
        self.clean_chars = re.compile(r'[^A-Za-züöäÖÜÄß ]', re.MULTILINE)
        self.clean_http_urls = re.compile(r'https*\\S+', re.MULTILINE)
        self.clean_at_mentions = re.compile(r'@\\S+', re.MULTILINE)

    def predict_sentiment(self, texts):
        texts = [self.clean_text(text) for text in texts]
        # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
        encoded = self.tokenizer.batch_encode_plus(texts,padding=True, add_special_tokens=True,truncation=True, return_tensors="pt")
        encoded = encoded.to(self.device)
        with torch.no_grad():
                logits = self.model(**encoded)
        
        label_ids = torch.argmax(logits[0], axis=1)
        return [self.model.config.id2label[label_id.item()] for label_id in label_ids]

    # def replace_numbers(self,text: str) -> str:
    #         return text.replace("0"," null").replace("1"," eins").replace("2"," zwei").replace("3"," drei").replace("4"," vier").replace("5"," fünf").replace("6"," sechs").replace("7"," sieben").replace("8"," acht").replace("9"," neun")         

    def clean_text(self, text):
            text = text.replace("\n", " ")        
            text = self.clean_http_urls.sub('',text)
            text = self.clean_at_mentions.sub('',text)        
            #text = self.replace_numbers(text)                
            text = self.clean_chars.sub('', text) # use only text chars                          
            text = ' '.join(text.split()) # substitute multiple whitespace with single whitespace   
            text = text.strip().lower()
            return text

# texts = ["Mit keinem guten Ergebniss","Das war unfair", "Das ist gar nicht mal so gut",
#         "Total awesome!","nicht so schlecht wie erwartet", "Das ist gar nicht mal so schlecht",
#         "Der Test verlief positiv.","Sie fährt ein grünes Auto.", "Der Fall wurde an die Polzei übergeben."]

model = SentimentModel(model_name = "oliverguhr/german-sentiment-bert")

#print(model.predict_sentiment(texts))

c:\MeineProgramme\Anaconda\envs\bdcc\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Daten müssen gesplittet werden, um in den Speicher zu passen
def split_dataframe(df, chunk_size = 1000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [11]:
from IPython.display import clear_output

# Durchlaufen des Modells
ergebnisliste = []
laenge = len(df)
chunksize = 50
for i, x in enumerate(split_dataframe(df, chunksize)):
    clear_output(wait=True)
    print("Prozent: ", (i+1)*chunksize / laenge)
    print("Genau: ", (i+1)*chunksize, " / ", laenge)
    ergebnisliste.append(model.predict_sentiment(x["tweet"]))


Prozent:  1.0324483775811208
Genau:  1050  /  1017


In [12]:
import itertools
newListTMP = list(itertools.chain(*ergebnisliste))

In [13]:
df["Sentiment"] = newListTMP

In [14]:
# Mapping
def mapsenti(x):
    if x == "negative":
        return -1
    if x == "positive":
        return 1
    else:
        return 0

In [15]:
df["BertSentimentNumber"] = df["Sentiment"].apply(lambda x: mapsenti(x))

In [18]:
from textblob_de import TextBlobDE

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hahnb\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [25]:
text = '''
"Der Blob" macht in seiner unbekümmert-naiven Weise einfach nur Spass.
Er hat eben den gewissen Charme, bei dem auch die eher hölzerne Regie und
das konfuse Drehbuch nicht weiter stören. Ich hasse mein Leben.
'''

blob = TextBlobDE(text)

In [26]:
for sentence in blob.sentences:
    print(sentence.sentiment.polarity)

1.0
0.0
0.0


# Naive Bayes trainieren
https://textblob-de.readthedocs.io/en/latest/api_reference.html#module-textblob_de.sentiments